# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124039e+02     1.534782e+00
 * time: 0.6949489116668701
     1     1.066313e+01     8.958295e-01
 * time: 3.0241658687591553
     2    -1.215090e+01     9.777192e-01
 * time: 3.6448588371276855
     3    -3.423560e+01     7.846238e-01
 * time: 4.583132028579712
     4    -4.764398e+01     6.167606e-01
 * time: 5.505109071731567
     5    -5.693205e+01     2.084927e-01
 * time: 6.527343988418579
     6    -5.978892e+01     1.392994e-01
 * time: 7.209388017654419
     7    -6.092089e+01     5.084215e-02
 * time: 7.858442068099976
     8    -6.137536e+01     5.442510e-02
 * time: 8.828134059906006
     9    -6.161386e+01     4.332293e-02
 * time: 9.447572946548462
    10    -6.181589e+01     2.473313e-02
 * time: 10.0709969997406
    11    -6.195942e+01     1.854885e-02
 * time: 10.69803786277771
    12    -6.203367e+01     1.479262e-02
 * time: 11.330141067504883
    13    -6.209054e+01     1.629012e-02
 * time: 11.965405941009521
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557691
    AtomicNonlocal      14.8522652
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485377 
    Xc                  -19.3336822

    total               -62.261666462485
